# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [166]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [167]:
# your code here
orders = pd.read_csv('Orders.csv')

orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [168]:
orders.amount_spent.unique()

array([ 15.3 ,  20.34,  22.  , ...,  25.08, 469.44,  66.36])

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

to solve our problem we first drop the unnamed column

In [169]:
# your code here

data = orders.drop(['Unnamed: 0'], axis = 1)
data

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


for unique Customer ID we pivot the table using the index as customer ID, and sum their total spent in the aggregate function

In [170]:
# sub problem 1
df = data.pivot_table(index= ['CustomerID'], values = ['amount_spent'], aggfunc = {'amount_spent':'sum'}).reset_index()
df.head()

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [171]:
# sub problem 2

display(df['amount_spent'].max())
display(df['amount_spent'].min())
print('size of bins = ', (df['amount_spent'].max()-df['amount_spent'].min())/4)
size = (df['amount_spent'].max()-df['amount_spent'].min())/4

df.amount_spent.sort_values(ascending = False).head(10)

280206.02

0.0

size of bins =  70051.505


1690    280206.02
4202    259657.30
3729    194550.79
3009    168472.50
1880    143825.06
55      124914.53
1334    117379.63
3772     91062.38
2703     81024.84
0        77183.60
Name: amount_spent, dtype: float64

2 alternatives here, we can let the programme quartile as he wants, or we can choose the size of the bins to cut and distinguish or customers.

In [172]:
binnames = ["regular", "Medium", "Preferred", "VIP"]
bins = pd.cut(df['amount_spent'],4, labels = binnames)
bins.value_counts()

regular      4329
Medium          5
Preferred       3
VIP             2
Name: amount_spent, dtype: int64

In [173]:
bins1 = pd.cut(df['amount_spent'],[0,70051.505, (70051.505*2), (70051.505*3), (70051.505*4)], labels = binnames)
bins1.value_counts()

regular      4328
Medium          5
Preferred       3
VIP             2
Name: amount_spent, dtype: int64

In [174]:
# sub problem 3

# df['VIP'] = (df['amount_spent'] < (df['amount_spent'].max()-df['amount_spent'].min()*4)) & (df['amount_spent'] > (df['amount_spent'].max()-df['amount_spent'].min()*3))
df['VIP'] = df['amount_spent'] > size*3
df['VIP']
display(df.VIP.sort_values(ascending = False).head())
# df[['amount_spent', 'VIP']]

df['Preferred'] = (df['amount_spent'] > size*2) & (df['amount_spent'] < size*3)
# display(df['Preferred'])
display(df.Preferred.sort_values(ascending = False).head())


4202     True
1690     True
2890    False
2896    False
2895    False
Name: VIP, dtype: bool

3729     True
3009     True
1880     True
0       False
2891    False
Name: Preferred, dtype: bool

In [175]:
def rank(x):
    if x > size*3:
        return 'VIP'
    elif (x < size*3) & (x > size*2):
        return 'Preferred'
    else:
        return 'Not preferred or VIP'

df['Ranked'] = list(map(rank, df['amount_spent']))
df['Ranked']

0       Not preferred or VIP
1       Not preferred or VIP
2       Not preferred or VIP
3       Not preferred or VIP
4       Not preferred or VIP
                ...         
4334    Not preferred or VIP
4335    Not preferred or VIP
4336    Not preferred or VIP
4337    Not preferred or VIP
4338    Not preferred or VIP
Name: Ranked, Length: 4339, dtype: object

Customers labeled according to the amount spent 

In [176]:
df.nlargest(10, ['amount_spent'])

,CustomerID,amount_spent,VIP,Preferred,Ranked
1690,14646,280206.02,True,False,VIP
4202,18102,259657.30,True,False,VIP
3729,17450,194550.79,False,True,Preferred
3009,16446,168472.50,False,True,Preferred
1880,14911,143825.06,False,True,Preferred
55,12415,124914.53,False,False,Not preferred or VIP
1334,14156,117379.63,False,False,Not preferred or VIP
3772,17511,91062.38,False,False,Not preferred or VIP
2703,16029,81024.84,False,False,Not preferred or VIP
0,12346,77183.60,False,False,Not preferred or VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [177]:
# your code here
# orders.groupby(['CustomerID', 'Country']).agg({'amount_spent':sum})

# df.groupby(['CustomerID', 'VIP']).agg({'amount_spent': sum, 'VIP': True})

final_orders = pd.merge(orders, df, how = 'inner',
                                    left_on = "CustomerID", 
                                    right_on = "CustomerID")

final_orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent_x,amount_spent_y,VIP,Preferred,Ranked
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,False,False,Not preferred or VIP
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,False,False,Not preferred or VIP
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,False,False,Not preferred or VIP
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,False,False,Not preferred or VIP
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,False,False,Not preferred or VIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541801,581578,22993,2011,12,5,12,set of 4 pantry jelly moulds,12,2011-12-09 12:16:00,1.25,12713,Germany,15.00,848.55,False,False,Not preferred or VIP
397920,541802,581578,22907,2011,12,5,12,pack of 20 napkins pantry design,12,2011-12-09 12:16:00,0.85,12713,Germany,10.20,848.55,False,False,Not preferred or VIP
397921,541803,581578,22908,2011,12,5,12,pack of 20 napkins red apples,12,2011-12-09 12:16:00,0.85,12713,Germany,10.20,848.55,False,False,Not preferred or VIP
397922,541804,581578,23215,2011,12,5,12,jingle bell heart antique silver,12,2011-12-09 12:16:00,2.08,12713,Germany,24.96,848.55,False,False,Not preferred or VIP


In [178]:
final_orders[['CustomerID', 'Ranked', ]]

,CustomerID,Ranked
0,17850,Not preferred or VIP
1,17850,Not preferred or VIP
2,17850,Not preferred or VIP
3,17850,Not preferred or VIP
4,17850,Not preferred or VIP
...,...,...
397919,12713,Not preferred or VIP
397920,12713,Not preferred or VIP
397921,12713,Not preferred or VIP
397922,12713,Not preferred or VIP


In [187]:
fo1 = final_orders.groupby(['Country', 'Ranked']).agg({'amount_spent_x':sum, 'CustomerID': pd.Series.nunique})
fo1

,,amount_spent_x,CustomerID
Country,Ranked,,
Australia,Not preferred or VIP,138521.310,9
Austria,Not preferred or VIP,10198.680,11
Bahrain,Not preferred or VIP,548.400,2
Belgium,Not preferred or VIP,41196.340,25
Brazil,Not preferred or VIP,1143.600,1
Canada,Not preferred or VIP,3666.380,4
Channel Islands,Not preferred or VIP,20450.440,9
Cyprus,Not preferred or VIP,13590.380,8
Czech Republic,Not preferred or VIP,826.740,1


from the dataframe above, we identified that 2 countries have the 2 VIP. The UK has one VIP and the netherlands have one as well, must be Jan ....

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [193]:
# your code here
# final_orders.groupby(['Ranked', 'Country']).agg({'amount_spent_x':sum, 'CustomerID': pd.Series.nunique})

final_orders.pivot_table(index = ['Ranked', 'Country'], values = ['amount_spent_x', 'CustomerID'], aggfunc = {'amount_spent_x':'sum', 'CustomerID': pd.Series.nunique})

CustomerID  amount_spent_x
Ranked               Country                                         
Not preferred or VIP Australia                      9      138521.310
                     Austria                       11       10198.680
                     Bahrain                        2         548.400
                     Belgium                       25       41196.340
                     Brazil                         1        1143.600
                     Canada                         4        3666.380
                     Channel Islands                9       20450.440
                     Cyprus                         8       13590.380
                     Czech Republic                 1         826.740
                     Denmark                        9       18955.340
                     EIRE                           2      121720.840
                     European Community             1        1300.250
                     Finland                       12       22546.080
                     France                        87      209024.050
                     Germany                       94      228867.140
                     Greece                         4        4760.520
                     Iceland                        1        4310.000
                     Israel                         3        7221.690
                     Italy                         14       17483.240
                     Japan                          8       37416.370
                     Lebanon                        1        1693.880
                     Lithuania                      1        1661.060
                     Malta                          2        2725.590
                     Netherlands                    8        5240.320
                     Norway                        10       36165.440
                     Poland                         6        7334.650
                     Portugal                      19       33439.890
                     RSA                            1        1002.310
                     Saudi Arabia                   1         145.920
                     Singapore                      1       21279.290
                     Spain                         30       61577.110
                     Sweden                         8       38378.330
                     Switzerland                   21       56443.950
                     USA                            4        3580.390
                     United Arab Emirates           2        1902.280
                     United Kingdom              3918     6685710.964
                     Unspecified                    4        2667.070
Preferred            EIRE                           1      143825.060
                     United Kingdom                 2      363023.290
VIP                  Netherlands                    1      280206.020
                     United Kingdom                 1      259657.300

In the same way we can identifiy the countries that have the preferred or the VIP.

EIRE haas 1 preferred
UK has 2 preferred and 1 VIP
Netherland has the only one dutch that counts !!!